### Training


In [1]:
%load_ext tensorboard

import random
import shutil
import os


import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
from sklearn.model_selection import train_test_split
import datetime
import json
import numpy as np
import seaborn as sn
import librosa
import math


2023-12-18 22:29:54.157156: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-18 22:29:55.232732: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [10]:
DATA_PATH = "data.json"

VALIDATION_SPLIT = 0.2 # percentage of dataset
TEST_SPLIT = 0.1 # percentage of dataset

NUM_SESSION_GROUPS = 10
LOGDIR = "logs/hparam_tuning/"
NUM_EPOCHS = 30
BATCH_SIZE = 32


In [3]:
# load training data from json file

with open(DATA_PATH, "r") as f:
    data = json.load(f)

x = np.array(data["mfcc"])
y = np.array(data["labels"])


In [4]:
# split data into train, validation and test sets

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SPLIT)
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=VALIDATION_SPLIT)

# add an axis to input sets to match the shape CNN expects (last axis is like channel in color images)
x_train = x_train[..., np.newaxis]
x_test = x_test[..., np.newaxis]
x_validation = x_validation[..., np.newaxis]


In [5]:
# choose hyperparameters to tune

# HP_CONV_LAYERS = hp.HParam("conv_layers", hp.IntInterval(1, 3))
# HP_CONV_KERNEL_SIZE = hp.HParam("conv_kernel_size", hp.Discrete([3, 5]))
# HP_POOL_SIZE = hp.HParam("conv_pool_size", hp.Discrete([2, 3]))
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([32, 64, 96]))
HP_DENSE_LAYERS = hp.HParam("dense_layers", hp.IntInterval(1, 3))
HP_DROPOUT = hp.HParam("dropout", hp.RealInterval(0.2, 0.5))
HP_OPTIMIZER = hp.HParam("optimizer", hp.Discrete(["adam", "sgd"]))

HPARAMS = [
    # HP_CONV_LAYERS,
    # HP_CONV_KERNEL_SIZE,
    # HP_POOL_SIZE,
    HP_NUM_UNITS,
    HP_DENSE_LAYERS,
    HP_DROPOUT,
    HP_OPTIMIZER,
]

METRICS = [
    hp.Metric(
        "epoch_accuracy",
        group="validation",
        display_name="accuracy (val)",
    ),
    hp.Metric(
        "epoch_loss",
        group="validation",
        display_name="loss (val)",
    ),
    hp.Metric(
        "batch_accuracy",
        group="train",
        display_name="accuracy (train)",
    ),
    hp.Metric(
        "batch_loss",
        group="train",
        display_name="loss (train)",
    ),
]

# with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
#     hp.hparams_config(
#         hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
#         metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
#     )


In [6]:
def create_model(hparams, seed):
    rng = random.Random(seed)

    INPUT_SHAPE = (x_train.shape[1], x_train.shape[2], 1)

    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape=INPUT_SHAPE))

   #  conv_filters = 8
   #  for _ in range(hparams[HP_CONV_LAYERS]):
   #     model.add(
   #        tf.keras.layers.Conv2D(
   #           filters=conv_filters,
   #          #  kernel_size=hparams[HP_CONV_KERNEL_SIZE],
   #           kernel_size=3,
   #           padding="same",
   #           activation="relu"
   #        )
   #     )
   #     model.add(
   #        tf.keras.layers.MaxPooling2D(
   #           pool_size=hparams[HP_POOL_SIZE],
   #           strides=hparams[HP_POOL_SIZE]-1,
   #           padding="same"
   #        )
   #     )
   #     model.add(tf.keras.layers.BatchNormalization())
   #     conv_filters *= 2
    
    # 1st conv layer
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=INPUT_SHAPE))
    model.add(tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())

    # 2nd conv layer
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())

    # 3rd conv layer\n",
    model.add(tf.keras.layers.Conv2D(32, (2, 2), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    
    # flatten output and feed it into dense layer
    model.add(tf.keras.layers.Flatten())

    for _ in range(hparams[HP_DENSE_LAYERS]):
        model.add(tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation="relu"))

    model.add(tf.keras.layers.Dropout(hparams[HP_DROPOUT], seed=rng.random()))

    # output layer
    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    model.compile(
       optimizer=hparams[HP_OPTIMIZER],
       loss='sparse_categorical_crossentropy',
       metrics=['accuracy']
    )

    return model

#     fit_log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#     tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=fit_log_dir, histogram_freq=1)

#     model.fit(x=x_train,
#               y=y_train,
#               validation_data=(x_validation, y_validation),
#               batch_size=32,
#               epochs=30,
#               callbacks=[tensorboard_callback])
    
#     _, accuracy = model.evaluate(x_test, y_test)
#     return accuracy


# def run(run_dir, hparams):
#   with tf.summary.create_file_writer(run_dir).as_default():
#     hp.hparams(hparams)  # record the values used in this trial
#     accuracy = train_model(hparams)
#     tf.summary.scalar(METRIC_ACCURACY, accuracy)

In [7]:
def run(base_logdir, session_id, hparams):
    model = create_model(hparams=hparams, seed=session_id)
    logdir = os.path.join(base_logdir, session_id)

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=logdir,
        histogram_freq=1
    )

    hparams_callback = hp.KerasCallback(logdir, hparams)

    model.fit(
        x=x_train,
        y=y_train,
        epochs=NUM_EPOCHS,
        batch_size=BATCH_SIZE,
        shuffle=False,
        validation_data=(x_validation, y_validation),
        callbacks=[tensorboard_callback, hparams_callback]
    )


In [11]:
def run_all(logdir, verbose=False):
    rng = random.Random(0)

    with tf.summary.create_file_writer(logdir).as_default():
        hp.hparams_config(hparams=HPARAMS, metrics=METRICS)

    sessions_per_group = 2
    num_sessions = NUM_SESSION_GROUPS * sessions_per_group
    session_index = 0  # across all session groups
    for group_index in range(NUM_SESSION_GROUPS):
        hparams = {h: h.domain.sample_uniform(rng) for h in HPARAMS}
        hparams_string = str(hparams)
        for repeat_index in range(sessions_per_group):
            session_id = str(session_index)
            session_index += 1
            if verbose:
                print(
                    "--- Running training session %d/%d"
                    % (session_index, num_sessions)
                )
                print(hparams_string)
                print("--- repeat #: %d" % (repeat_index + 1))
            run(
                base_logdir=logdir,
                session_id=session_id,
                hparams=hparams,
            )


In [12]:
# main

np.random.seed(0)
logdir = LOGDIR
shutil.rmtree(logdir, ignore_errors=True)
print(f"Saving output to {logdir}")
run_all(logdir=logdir, verbose=True)
print(f"Done. Output saved to {logdir}")


Saving output to logs/hparam_tuning/
--- Running training session 1/20
{HParam(name='num_units', domain=Discrete([32, 64, 96]), display_name=None, description=None): 64, HParam(name='dense_layers', domain=IntInterval(1, 3), display_name=None, description=None): 2, HParam(name='dropout', domain=RealInterval(0.2, 0.5), display_name=None, description=None): 0.21214531345423326, HParam(name='optimizer', domain=Discrete(['adam', 'sgd']), display_name=None, description=None): 'sgd'}
--- repeat #: 1
Epoch 1/30
212/212 [==============================] - 4s 15ms/step - loss: 1.9205 - accuracy: 0.3137 - val_loss: 1.6638 - val_accuracy: 0.4303
Epoch 2/30
212/212 [==============================] - 3s 14ms/step - loss: 1.3405 - accuracy: 0.5306 - val_loss: 1.2647 - val_accuracy: 0.5502
Epoch 3/30
212/212 [==============================] - 3s 13ms/step - loss: 1.0809 - accuracy: 0.6235 - val_loss: 1.2232 - val_accuracy: 0.5798
Epoch 4/30
212/212 [==============================] - 3s 12ms/step - loss

2023-12-18 22:42:53.826752: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


212/212 [==============================] - 6s 15ms/step - loss: 1.7006 - accuracy: 0.3978 - val_loss: 1.6409 - val_accuracy: 0.4527
Epoch 2/30
212/212 [==============================] - 3s 14ms/step - loss: 1.0914 - accuracy: 0.6401 - val_loss: 1.1145 - val_accuracy: 0.6348
Epoch 3/30
212/212 [==============================] - 3s 14ms/step - loss: 0.7856 - accuracy: 0.7363 - val_loss: 0.9014 - val_accuracy: 0.7258
Epoch 4/30
212/212 [==============================] - 3s 13ms/step - loss: 0.6120 - accuracy: 0.8011 - val_loss: 0.8186 - val_accuracy: 0.7630
Epoch 5/30
212/212 [==============================] - 3s 13ms/step - loss: 0.4445 - accuracy: 0.8599 - val_loss: 0.9683 - val_accuracy: 0.7411
Epoch 6/30
212/212 [==============================] - 3s 14ms/step - loss: 0.3520 - accuracy: 0.8890 - val_loss: 0.9209 - val_accuracy: 0.7624
Epoch 7/30
212/212 [==============================] - 3s 14ms/step - loss: 0.2966 - accuracy: 0.9093 - val_loss: 0.8883 - val_accuracy: 0.7790
Epoch 8/30

In [10]:
%tensorboard --logdir logs/hparam_tuning

In [ ]:
%tensorboard --logdir logs/fit